In [1]:
from torchvision.models import resnet50
import torch

from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import numpy as np
import cv2

from torch import optim
from tqdm import tqdm

import json
import pickle

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        myDict = pickle.load(fo, encoding='latin1')
    return myDict

In [3]:
metaData = unpickle('./cifar-10-batches-py/batches.meta')
labels=metaData['label_names']

DICT={}
idx=0
for i in labels:
    DICT[i]=idx
    idx+=1

In [4]:
model=resnet50(pretrained=True)

# for param in model.parameters():
#     param.requires_grad=False

model.fc=nn.Sequential(*[
    nn.Linear(in_features=2048, out_features=10),
])

/home/chan/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/chan/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
class Dataset(Dataset):
    def __init__(self, json_path, transform=None):
        self.transform=transform
        with open(json_path,'r') as f:
            self.json_data=json.load(f)
        
        self.label=[]
        self.data=[]

        for value in self.json_data:
            self.data.append(value)
            self.label.append(DICT[str(value.split('/')[3])])

    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, index):
        image_path=self.data[index]
        image=cv2.imread(image_path)
        image=image.transpose(2,0,1)
        label=self.label[index]
        

        return image,label

In [8]:
TRAIN_PATH="./jsons/train_truck_x.json"
NUM_BATCH=64
EPOCHS=50
LEARNING_RATE=5e-4
DEVICE="cuda:0" 

In [9]:
train_data=Dataset(TRAIN_PATH)
train_size=len(train_data)

train_dataset, validation_dataset = random_split(train_data, [int(train_size*0.9),train_size-int(train_size*0.9)])

train_dataloader=DataLoader(train_dataset,batch_size=NUM_BATCH)
validation_dataloader=DataLoader(validation_dataset,batch_size=NUM_BATCH)

In [10]:
def validate(model, data):
    total =0
    correct=0

    with torch.no_grad():
        for(images, labels) in data:
            images=images.type(torch.cuda.FloatTensor)
            images=images.to(DEVICE)
            labels=labels.to(DEVICE)
            x=model(images)
            pred=torch.argmax(x,1)
            total += x.size(0)
            correct += torch.sum(pred==labels)
            
    return correct*100/total

In [11]:
def train(num_epoch=EPOCHS, lr=LEARNING_RATE, device=DEVICE):
    now_model=model.to(device)
    cel=nn.CrossEntropyLoss()
    optimizer=optim.Adam(now_model.parameters(),lr=lr,weight_decay=0)

    max_accuracy=0
    
    for epoch in range(num_epoch):
        for _, (images,labels) in tqdm(enumerate(train_dataloader)):
            if _ ==epoch:
                hi=images[0].detach().cpu().numpy()
                hi=hi.transpose(1,2,0)
                cv2.imwrite("./hi.png",hi)

            images=images.type(torch.cuda.FloatTensor)
            images=images.to(device)
            labels=labels.to(device)

            optimizer.zero_grad()
            pred=now_model(images)
            loss=cel(pred,labels)  
            loss.backward()
            optimizer.step()

        print("\n\n==================Let's validation!!==================")
        accuracy=float(validate(now_model,validation_dataloader))
        print("Epoch: ",epoch+1,"Accuracy: ",accuracy,"%","   loss : ",loss.item())
        
        if accuracy>max_accuracy:
            torch.save(now_model,'HOPE.pt')
            max_accuracy=accuracy
            print("find best!")

In [12]:
train()

633it [00:29, 21.79it/s]




==================Let's validation!!==================
Epoch:  1 Accuracy:  76.42222595214844 %    loss :  0.5730442404747009
find best!


633it [00:28, 22.48it/s]




==================Let's validation!!==================
Epoch:  2 Accuracy:  78.24444580078125 %    loss :  0.41422611474990845
find best!


633it [00:27, 22.95it/s]


...



==================Let's validation!!==================
Epoch:  48 Accuracy:  82.64444732666016 %    loss :  0.00026628177147358656


633it [00:22, 28.77it/s]




==================Let's validation!!==================
Epoch:  49 Accuracy:  82.02222442626953 %    loss :  0.0663137212395668


633it [00:22, 27.76it/s]




==================Let's validation!!==================
Epoch:  50 Accuracy:  82.42222595214844 %    loss :  0.0012002082075923681
